In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
import os
os.chdir("/content/drive/My Drive/Colab Notebooks/GAN")

In [3]:
# Imports
from keras.optimizers import RMSprop
from keras.datasets import mnist
from keras.layers import Dense, Reshape, Conv2DTranspose, BatchNormalization
from keras.layers import Activation, LeakyReLU, Conv2D, Flatten, Input
from keras.models import Model
import numpy as np
from matplotlib import pyplot as plt
import os

Using TensorFlow backend.


In [4]:
'''Dynamic and Hyper parameters'''
latent_dim = 100
image_size = 28
image_shape = (image_size, image_size, 1)

gen_filters = [128, 64, 32, 1]
gen_strides = [2, 2, 1, 1]

dis_filters = [32, 64, 128, 256]
dis_strides = [2, 2, 2, 1]

kernel_size = (5, 5)
alpha = 0.2

dis_lr = 2e-4
dis_decay = 6e-8
dis_opt = RMSprop(lr = dis_lr, decay = dis_decay)

adv_lr = dis_lr * 0.5
adv_decay = dis_decay * 0.5
adv_opt = RMSprop(lr = adv_lr, decay = adv_decay)

test_size = 16
batch_size = 64
epochs = 40000

save_intervals = 500
save_adress = "./"

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [5]:
'''Download dataset'''
(x_train, _), (x_test, _) = mnist.load_data()
x_train = x_train.reshape(-1, image_size, image_size, 1).astype("float32") / 255

In [6]:
'''BatchNormalization-Relu is used repeatedly so get it into a function'''
def BN_RELU(x):

    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    return x

In [7]:
def build_generator(inputs, image_size = 28):

    image_size = image_size // 4

    x = inputs
    x = Dense(image_size * image_size * gen_filters[0])(x)
    x = Reshape((image_size, image_size, gen_filters[0]))(x)

    for stride, filter in zip(gen_strides, gen_filters):
        x = BN_RELU(x)
        x = Conv2DTranspose(filters = filter,
                            strides = stride,
                            padding = "same",
                            kernel_size = kernel_size)(x)
    outputs = Activation("sigmoid")(x)

    model = Model(inputs, outputs, name = "Generator")
    model.summary()
    return model

In [8]:
def build_discriminator(inputs):

    x = inputs

    for stride, filter in zip(dis_strides, dis_filters):
        x = LeakyReLU(alpha = alpha)(x)
        x = Conv2D(filters = filter,
                   strides = stride,
                   padding = "same",
                   kernel_size = kernel_size)(x)
    x = Flatten()(x)
    x = Dense(1)(x)
    outputs = Activation("sigmoid")(x)

    model = Model(inputs, outputs, name = "Discriminator")
    model.summary()
    return model

In [9]:
def train(models):

    gen, dis, adv = models
    test_noise = np.random.uniform(low = -1, high = 1, size =(test_size, latent_dim))

    for epoch in range(1, epochs + 1): # (+ 1) for running plot function
        random_indices = np.random.randint(low = 0, high = x_train.shape[0], size = batch_size)
        real_imgs = x_train[random_indices]
        real_labels = np.ones(shape = (batch_size, 1))

        z = np.random.uniform(low = -1, high = 1, size =(batch_size, latent_dim))
        fake_imgs = gen.predict(z)
        fake_labels = np.zeros(shape = (batch_size, 1))

        dis_x = np.concatenate([real_imgs, fake_imgs])
        dis_y = np.concatenate([real_labels, fake_labels])

        loss, acc = dis.train_on_batch(dis_x, dis_y)

        log = "step:{} dis[loss:{:3f} acc:{:3f}]".format(epoch, loss, acc)

        adv_x = np.random.uniform( low=-1, high=1, size=(batch_size, latent_dim) )
        adv_y = np.ones(shape = (batch_size, 1))

        loss, acc = adv.train_on_batch(adv_x, adv_y)

        log += " adv[loss:{:3f} acc:{:3f}]".format( loss, acc )

        print(log)

        if epoch % save_intervals == 0:
            plot_image(gen, test_noise, epoch)

In [10]:
"""Show what is being produced"""
def plot_image(generator, input_noise, step, show = False):

    z = input_noise
    n_images = z.shape[0]
    images = generator.predict(z)

    rows = np.sqrt(n_images)

    plt.figure(figsize=(2,2))

    for i in range(n_images):
        plt.subplot(rows, rows, i+1)
        plt.imshow(images[i].reshape(image_size, image_size), cmap="gray")
        plt.axis("off")
    plt.savefig(os.path.join(save_adress, "step{}.png".format(step)))

    if show:
        plt.show()
    else:
        plt.close("all")

In [11]:
dis_inputs = Input(shape = image_shape)
dis = build_discriminator(dis_inputs)
dis.compile(optimizer = dis_opt,
            loss = "binary_crossentropy",
            metrics = ["acc"])

gen_inputs = Input(shape = (latent_dim,))
gen = build_generator(inputs = gen_inputs)

dis.trainable = False
adv_inputs = gen_inputs
adv_outputs = dis(gen(adv_inputs))
adv = Model(adv_inputs, adv_outputs, name="Adversarial")
adv.summary()
adv.compile(optimizer = adv_opt,
            loss = "binary_crossentropy",
            metrics = ["acc"])

models = gen, dis, adv
train(models)

Model: "Discriminator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 32)        832       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          51264     
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 128)       

/tensorflow-1.15.2/python3.6/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


step:1 dis[loss:0.694080 acc:0.453125] adv[loss:0.840461 acc:0.000000]
step:2 dis[loss:0.608813 acc:0.992188] adv[loss:0.875904 acc:0.000000]
step:3 dis[loss:0.483240 acc:1.000000] adv[loss:0.911923 acc:0.000000]
step:4 dis[loss:0.334150 acc:1.000000] adv[loss:0.396392 acc:1.000000]


/tensorflow-1.15.2/python3.6/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Streaming output truncated to the last 5000 lines.
step:35002 dis[loss:0.582155 acc:0.718750] adv[loss:0.984018 acc:0.375000]
step:35003 dis[loss:0.546189 acc:0.718750] adv[loss:1.243940 acc:0.234375]
step:35004 dis[loss:0.513094 acc:0.742188] adv[loss:1.186409 acc:0.312500]
step:35005 dis[loss:0.545860 acc:0.679688] adv[loss:1.621131 acc:0.046875]
step:35006 dis[loss:0.507916 acc:0.742188] adv[loss:0.895848 acc:0.359375]
step:35007 dis[loss:0.493421 acc:0.718750] adv[loss:1.575918 acc:0.093750]
step:35008 dis[loss:0.636445 acc:0.664062] adv[loss:1.201162 acc:0.296875]
step:35009 dis[loss:0.459336 acc:0.773438] adv[loss:1.349894 acc:0.203125]
step:35010 dis[loss:0.562066 acc:0.671875] adv[loss:1.079811 acc:0.265625]
step:35011 dis[loss:0.520909 acc:0.750000] adv[loss:1.563793 acc:0.125000]
step:35012 dis[loss:0.609722 acc:0.656250] adv[loss:0.961679 acc:0.453125]
step:35013 dis[loss:0.460159 acc:0.781250] adv[loss:1.281279 acc:0.281250]
step:35014 dis[loss:0.487687 acc:0.773438] adv[lo